In [1]:
import codecs
import os
import sys
import time
		

In [2]:
tags=[]

In [3]:

def max_connect(x, y, viterbi_matrix, emission, transmission_matrix):
	max = -99999
	path = -1
	for k in range(len(tags)):
		val = viterbi_matrix[k][x-1] * transmission_matrix[k][y]
		if val * emission > max:
			max = val
			path = k
	return max, path

In [4]:

filepath = "tamil_training.txt"
language = "tamil"
exclude = ["<s>", "</s>", "START", "END"]
wordtypes = []
tagscount = []
	# Open training file to read the contents
f = codecs.open(filepath, 'r', encoding='utf-8')
file_contents = f.readlines()
print(file_contents[:10])		#<-------------------------------------
print(len(file_contents))		#<-------------------------------------
count=0
for x in range(len(tags)):
	tagscount.append(0)
for x in range(len(file_contents)):
	line = file_contents.pop(0).strip().split('\\')
	

	for i, word in enumerate(line):  #for has 2 itertations the first the actual word (i=0) and then the tag (i=1)
		if i == 0:
			if word in exclude:
				count+=1
			if word not in wordtypes and word not in exclude:
				wordtypes.append(word)
		else:
			if word not in tags and word not in exclude:
				tags.append(word)
				tagscount.append(0)
				tagscount[tags.index(word)]+=1
			if word in tags and word not in exclude:
				tagscount[tags.index(word)] += 1
		
f.close()
print(tags)
print(len(wordtypes))	#<-------------------------------------
print(tagscount)
print(count//2)	

['<s>\\START\r\n', 'Published\\PROPN\r\n', 'on\\PROPN\r\n', '2015-11-19\\NUM\r\n', '11:02:15\\NUM\r\n', 'விண்வெளியில்\\NOUN\r\n', 'இருந்து\\ADP\r\n', 'இன்று\\ADV\r\n', 'முற்பகல்\\NOUN\r\n', '11.45\\NUM\r\n']
127799
['PROPN', 'NUM', 'NOUN', 'ADP', 'ADV', 'VERB', 'CCONJ', 'DET', 'PUNCT', 'ADJ', 'PRON', 'VAUX', 'PART', 'NEG', 'UT', 'WQ', 'INTF']
20667
[14037, 5552, 59040, 975, 4041, 15339, 2969, 1093, 1846, 4457, 4310, 884, 169, 83, 118, 14, 13]
6438


In [5]:
emission_matrix = []
transmission_matrix = []
	# Initialize emission matrix
for x in range(len(tags)):
	emission_matrix.append([])
	for y in range(len(wordtypes)):
		emission_matrix[x].append(0)
	# Initialize transmission matrix
for x in range(len(tags)):
	transmission_matrix.append([])
	for y in range(len(tags)):
		transmission_matrix[x].append(0)

In [6]:
f = codecs.open(filepath ,'r', encoding='utf-8')
file_contents = f.readlines()
print(file_contents[:10])		#<-------------------------------------
	# Update emission and transmission matrix with appropriate counts
row_id = -1
for x in range(len(file_contents)):
	line = file_contents.pop(0).strip().split('\\')
	if line[0] not in exclude:
		col_id = wordtypes.index(line[0])
		prev_row_id = row_id
		try:
			row_id = tags.index(line[1])
		except:
			print(x)
			break
		emission_matrix[row_id][col_id] += 1
		if prev_row_id != -1:
			transmission_matrix[prev_row_id][row_id] += 1
	else:
		row_id = -1

['<s>\\START\r\n', 'Published\\PROPN\r\n', 'on\\PROPN\r\n', '2015-11-19\\NUM\r\n', '11:02:15\\NUM\r\n', 'விண்வெளியில்\\NOUN\r\n', 'இருந்து\\ADP\r\n', 'இன்று\\ADV\r\n', 'முற்பகல்\\NOUN\r\n', '11.45\\NUM\r\n']


In [7]:
for x in range(len(tags)):
	for y in range(len(wordtypes)):
		if tagscount[x] != 0:
			emission_matrix[x][y] = float(emission_matrix[x][y]) / tagscount[x]
	# Divide each entry in transmission matrix by appropriate tag count to store probabilities in each entry instead of just count
for x in range(len(tags)):
	for y in range(len(tags)):
		if tagscount[x] != 0:
			transmission_matrix[x][y] = float(transmission_matrix[x][y]) / tagscount[x]

In [8]:
testpath = "tamil_testing_sentences.txt"
file_test = codecs.open(testpath, 'r', encoding='utf-8')
test_input = file_test.readlines()
print(len(test_input))
	# Declare variables for test words and pos tags
test_words = []
pos_tags = []
	# Create an output file to write the output tags for each sentences
file_output = codecs.open(language +"_tags.txt", 'w', 'utf-8')
file_output.close()
	# For each line POS tags are computed
for j in range(len(test_input)):
	test_words = []
	pos_tags = []
	line = test_input.pop(0).strip().split(' ')
	for word in line:
		test_words.append(word)
		pos_tags.append(-1)
	viterbi_matrix = []
	viterbi_path = []
		# Initialize viterbi matrix of size |tags| * |no of words in test sentence|
	for x in range(len(tags)):
		viterbi_matrix.append([])
		viterbi_path.append([])
		for y in range(len(test_words)):
			viterbi_matrix[x].append(0)
			viterbi_path[x].append(0)
		# Update viterbi matrix column wise
	for x in range(len(test_words)):
		for y in range(len(tags)):
			if test_words[x] in wordtypes:
				word_index = wordtypes.index(test_words[x])
				tag_index = tags.index(tags[y])
				emission = emission_matrix[tag_index][word_index]
			else:
				emission = 0.001
			if x > 0:
				max, viterbi_path[y][x] = max_connect(x, y, viterbi_matrix, emission, transmission_matrix)
			else:
				max = 1
			viterbi_matrix[y][x] = emission * max
		# Identify the max probability in last column i.e. best tag for last word in test sentence
	maxval = -999999
	maxs = -1
	for x in range(len(tags)):
		if viterbi_matrix[x][len(test_words)-1] > maxval:
			maxval = viterbi_matrix[x][len(test_words)-1]
			maxs = x
		# Backtrack and identify best tags for each words
	for x in range(len(test_words)-1, -1, -1):
		pos_tags[x] = maxs
		maxs = viterbi_path[maxs][x]
		# Display POS Tags in the console.
		# print pos_tags
		# Print output to the file.	
	file_output = codecs.open(language +"_tags.txt", 'a', 'utf-8')
	for i, x in enumerate(pos_tags):
		file_output.write(tags[x] + " ")
	file_output.write("\n")	
f.close()
file_output.close()
file_test.close()

print ("\nKindly check " + language + "_tags.txt file for POS tags.")

716

Kindly check tamil_tags.txt file for POS tags.
